# 坡位分类

In [1]:
import arcpy
from arcpy.sa import *
from arcpy import env
from arcpy.conversion import *
from arcpy.management import *
from arcpy.da import *
import math

In [2]:
# 分析函数

# 取消并行处理
def disable_parallel_processing(func):
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster)
    print("fill done!")
    return fill_raster
# 流向
@disable_parallel_processing
def flowdir_dem(dem_raster,dir_index):
    """"D8算法,dir_index{1:正常流动,0:强制外向流动}"""
    if dir_index:
        flowdir_raster = FlowDirection(dem_raster,"NORMAL",None,"D8")
        return flowdir_raster
    else:
        flowdir_raster = FlowDirection(dem_raster,"FORCE", None, "D8")
        return flowdir_raster
# 流量
@disable_parallel_processing
def acc_dem(fd_raster):
    """输入流向类型D8"""
    acc_raster = FlowAccumulation(fd_raster,None,"FLOAT","D8")
    return acc_raster

# 基于焦点的正负地形
def pos_neg_dem(dem_raster,focus_size,index_):
    """focus_size:统计焦点,Test:if:(true:1正),else:0负"""
    if index_:
        pos_raster = Test(dem_raster - FocalStatistics(dem_raster,NbrRectangle(focus_size,focus_size,"CELL"),"MEAN","DATA"),"value>0")
        return pos_raster
    else:
        neg_raster = Test(dem_raster - FocalStatistics(dem_raster,NbrRectangle(focus_size,focus_size,"CELL"),"MEAN","DATA"),"value<0")
        return neg_raster
# 基于指定值的负地形
def digital_dem(dem_raster,elevation_value):
    """基于指定值于DEM作差得出的栅格"""
    return elevation_value-arcpy.Raster(dem_raster)
# 坡度
@disable_parallel_processing
def get_slope(dem_raser):
    """取消并行处理"""
    slope_raster = Slope(dem_raser,"DEGREE",1,"PLANAR","METER")
    return slope_raster
# 坡向
@disable_parallel_processing
def get_asp(dem_raster):
    """取消并行处理"""
    asp_raster = Aspect(dem_raster,"PLANAR", "METER", "GEODESIC_AZIMUTHS")
    return asp_raster
# 计算曲率
@disable_parallel_processing
def get_cur(dem_raster,count):
    """count:{0:曲率,1:[剖面曲率,平面曲率]}"""
    if count:
        asp_raster = Curvature(dem_raster,1,"profile_curve","plan_curve")
        return asp_raster
    else:
        asp_raster = Curvature(dem_raster,1,None,None)
        return asp_raster
    



In [3]:
# 转换计算分析

# raster to polygon
def raster_to_polygon(raster,out_name):
    """raster to polygon ! Reduced surface!!!"""
    RasterToPolygon(raster, out_name, "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
    return None

# 得到多边形最小边界园直径
def calc_diameter(feature_file):
    # 默认字段名字为Diameter
    diameter_field = "Diameter"
    # 创建一个新的字段来存储直径值
    AddField(feature_file, diameter_field, "DOUBLE")
    # 使用游标遍历要素类中的要素，并计算小边界几何的直径
    with arcpy.da.UpdateCursor(feature_file, ["SHAPE@", diameter_field]) as cursor:
        for row in cursor:
            geometry = row[0]
            extent = geometry.extent
            diameter = math.sqrt(math.pow(extent.width,2)+math.pow(extent.height,2))
            row[1] = diameter
            cursor.updateRow(row)
# 消除多于面
def eli_polygon(element,out_name,min_area,min_length):
    """按条件消除面部件"""
    expression = f"Shape_Area <= {min_area} And Diameter <= {min_length}"
    temp_layer = MakeFeatureLayer(element)
    SelectLayerByAttribute(temp_layer,"NEW_SELECTION",expression)
    Eliminate(temp_layer,out_name,"LENGTH")
    return None
    
# 面转栅格

def polygon_to_raster(element,out_name,pixel_re):
    """pixel_re:像元参考,默认值字段为gridcode"""
    PolygonToRaster(element,"gridcode",out_name,"CELL_CENTER","NONE",pixel_re,"BUILD")
    return None

# 栅格合并
def calc_raster(raster_a,raster_b,method):
    """1:以raster_a为优先合并A和B两个栅格,0:以raster_a为优先合并A和B两个栅格,但不新增值"""
    # method 1
    raster_a = arcpy.Raster(raster_a)
    raster_b = arcpy.Raster(raster_b)
    if method:
        result_raster = Con(raster_a>0,raster_a,Con((raster_a>0)&(raster_b>0),7,raster_b))
        return result_raster
    # method 0
    else:
        result_raster = Con(raster_a>0,raster_a,raster_b)
        return result_raster

In [4]:
# 获取栅格数据窗口内的相对高程位置
def calc_dem_raster(dem_raster, focus_size):
    min_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "MINIMUM", "DATA"
    )
    max_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "MAXIMUM", "DATA"
    )
    mood_raster = max_raster - min_raster
    meet_raster = dem_raster - min_raster
    result_raster = Con(
        (((meet_raster / mood_raster) <= 0.25) | (mood_raster <= 0)),
        2,
        Con(
            (
                ((meet_raster / mood_raster) > 0.25)
                & ((meet_raster / mood_raster) <= 0.5)
            ),
            3,
            Con(
                (
                    ((meet_raster / mood_raster) > 0.5)
                    & ((meet_raster / mood_raster) <= 0.85)
                ),
                4,
                5,
            ),
        ),
    )
    return result_raster

In [5]:
# 山谷山脊线
def get_line(dem_raster,high_value,fcous_size,angle):
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        print("计算坡向坡度")
        z_asp = get_asp(dem_raster)
        z_slope = get_slope(z_asp)
        print("计算反地形")
        # fdx = RasterCalculator("fdx", f"3090 - {dem}")
        fdx = digital_dem(dem_raster,high_value)
        print("计算负地形坡向坡度")
        f_asp = get_asp(fdx)
        f_slope = get_slope(f_asp)
        print("计算坡度变化率")
        pd_rg = ((z_slope + f_slope) - Abs(z_slope - f_slope)) / 2

        print("焦点统计")
        mean_dem = FocalStatistics(dem_raster, NbrRectangle(fcous_size, fcous_size, "CELL"), "MEAN", "DATA")
        print("得到正负地形")

        zf_dem = dem_raster - mean_dem
        print("得到山脊线")
        sj_raster = Con((zf_dem > 0) & (pd_rg > angle), 6, 0)

        sg_raster = Con((zf_dem < 0) & (pd_rg > angle), 1, 0)

        return sj_raster, sg_raster


In [6]:
# 设定工作环境
env.workspace=r"D:\ArcgisData\pred_soildeep\pre_database\pred_soildeep_gz.gdb"
dem_file_path  = "DEM"

In [7]:
# 获取山脊山谷线
result_line = get_line(dem_file_path,4090,5,60)  # default 3090,3,70
result_line[0].save("SY_RIDGE_LINE")
result_line[1].save("SY_VALLEY_LINE")

计算坡向坡度
计算反地形
计算负地形坡向坡度
计算坡度变化率
焦点统计
得到正负地形
得到山脊线


In [ ]:
# 转面
raster_to_polygon("SY_RIDGE_LINE","SY_RIDGE_LINE_POLYGON")
raster_to_polygon("SY_VALLEY_LINE","SY_VALLEY_LINE_POLYGON")


In [ ]:
# 计算直径
calc_diameter("SY_RIDGE_LINE_POLYGON")
calc_diameter("SY_VALLEY_LINE_POLYGON")

In [ ]:
# 消除
eli_polygon("SY_RIDGE_LINE_POLYGON","SY_RIDGE_LINE_POLYGON_2000",2000,100)
eli_polygon("SY_VALLEY_LINE_POLYGON","SY_VALLEY_LINE_POLYGON_2000",2000,100)

In [ ]:
# 面转栅格
polygon_to_raster("SY_RIDGE_LINE_POLYGON_2000","SY_RIDGE_LINE_POLYGON_2000_RASTER","SY_DEM_5_DT_3")
polygon_to_raster("SY_VALLEY_LINE_POLYGON_2000","SY_VALLEY_LINE_POLYGON_2000_TASTER","SY_DEM_5_DT_3")

In [ ]:
#山脊和山谷栅格合并
calc_raster("SY_RIDGE_LINE_POLYGON_2000_RASTER","SY_VALLEY_LINE_POLYGON_2000_TASTER",1).save("SY_VALLEY_LINE_POLYGON_2000_TASTER_MERGE_RV")

In [8]:
# 计算相对位置
calc_dem_raster(dem_file_path,101).save("SY_DEM_RECLASS_101")

In [ ]:
# 栅格合并
calc_raster("SY_VALLEY_LINE_POLYGON_2000_TASTER_MERGE_RV","SY_DEM_RECLASS_101",0).save("SY_RESULT")